In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard, ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import numpy as np
from tqdm import tqdm
import os
from random import shuffle
from time import time
# tensorboard --logdir=logs/ --host localhost --port 8088

print(f'TensorFlow Version - {tf.__version__}')
print(f'Keras Version - {tf.keras.__version__}')

In [ ]:
from zipfile import ZipFile as zf
train_zip = zf('/kaggle/input/dogs-vs-cats/train.zip', 'r')
train_zip.extractall()
train_zip.close()
test_zip = zf('/kaggle/input/dogs-vs-cats/test1.zip', 'r')
test_zip.extractall()
test_zip.close()
IMG_SIZE = 50
LR = 0.0003
BATCH_SIZE = 32

In [ ]:
MODEL_NAME = 'cat_and_dog_LR-{}_MODEL-{}.h5'.format(LR,'CovNet-128(2)-64(2)-32(2)-512-128-1')
MODEL_PATH = os.path.join('saved_models',MODEL_NAME)

In [ ]:
TRAIN_DIR = './train/'
TEST_DIR = './test1/'

In [ ]:
def get_model(saved=True):
    '''
        Return the model
    '''
    if os.path.isfile(MODEL_PATH) and saved :
        print("Loading saved model {}".format(MODEL_NAME))
        return load_model(MODEL_PATH)
    
    # Declaring model
    model = Sequential()

    # 1st Block
    model.add(Conv2D(input_shape=(IMG_SIZE, IMG_SIZE, 1),filters=128, kernel_size=5, strides=1,padding='same',name = 'block1_conv1'))
    model.add(Conv2D(filters=128, kernel_size=5, strides=1,padding='same',name = 'block1_conv2'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2,name = 'block1_mxPool'))

    # 2nd Block
    model.add(Conv2D(filters=64, kernel_size=5, strides=1,padding='same',name = 'block2_conv1'))
    model.add(Conv2D(filters=64, kernel_size=5, strides=1,padding='same',name = 'block2_conv2'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2,name = 'block2_mxPool'))
    
    # 3rd Block
    model.add(Conv2D(filters=32, kernel_size=5, strides=1,padding='same',name = 'blk3_conv1'))
    model.add(Conv2D(filters=32, kernel_size=5, strides=1,padding='same',name = 'blk3_conv2'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2,name = 'block3_mxPool'))

    # 4th Block - FC Block
    dr_rate = 0.35
    model.add(Flatten(name = 'block4_flatten'))
    model.add(Dropout(dr_rate,name = 'block4_droupout1'))
    model.add(Dense(512, activation='relu',name = 'block4_dense1'))
    model.add(Dropout(dr_rate,name = 'block4_droupout2'))
    model.add(Dense(128, activation='relu',name = 'block4_dense2'))
    model.add(Dropout(dr_rate,name = 'block4_droupout3'))
    model.add(Dense(1, activation='sigmoid',name = 'block4_dense3'))

    return model

In [ ]:
def get_label(img):
    '''
        Return the label for images
    '''
    word = img.split('.')[0]
    if word == 'cat':
        return [0]
    else:
        return [1]

In [ ]:
def get_training_data():
    '''
        Return training data
    '''
    training_data = []
    if os.path.isfile('training_data_{}.npy'.format(IMG_SIZE)):
        return np.load('training_data_{}.npy'.format(IMG_SIZE))
    else:
        for img in tqdm(os.listdir(TRAIN_DIR)):
            label = get_label(img)
            path = os.path.join(TRAIN_DIR,img)
            img = cv2.resize(cv2.imread(path,cv2.IMREAD_GRAYSCALE), (IMG_SIZE,IMG_SIZE))
            img = img/255
            training_data.append([np.array(img),np.array(label)])
        shuffle(training_data)
        np.save('training_data_{}.npy'.format(IMG_SIZE),training_data)
        return np.array(training_data)

In [ ]:
def get_testing_data():
    '''
        Return testing data
    '''
    testing_data = []
    if os.path.isfile('testing_data_{}.npy'.format(IMG_SIZE)):
        return np.load('testing_data_{}.npy'.format(IMG_SIZE))
    else:
        for img in tqdm(os.listdir(TEST_DIR)):
            img_id = int(img.split('.')[0])
            path = os.path.join(TEST_DIR,img)
            img = cv2.resize(cv2.imread(path,cv2.IMREAD_GRAYSCALE), (IMG_SIZE,IMG_SIZE))
            img = img/255
            testing_data.append([np.array(img),img_id])
        testing_data.sort(key = lambda x: x[1])
        np.save('testing_data_{}.npy'.format(IMG_SIZE),testing_data)
        return np.array(testing_data)

# Preparation

In [ ]:
data = get_training_data()

partition = 1000             # Breaking -ve index
train = data[:-partition]    # For Training purpose
test= data[-partition:]      # For Validation purpose

# Training set
X_train = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
y_train = np.array([i[1] for i in train])

# Validation set
X_val = np.array([i[0] for i in test]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
y_val = np.array([i[1] for i in test])

In [ ]:
'''Effects added on image
    Rotation - ± 50 deegrees,
    Width Shift - ± 15 %
    Height Shift - ± 15 %
    Zoom - 30%
    Horizontal Flip
    Vertical Flip
'''
datagen = ImageDataGenerator(rotation_range=20,width_shift_range=0.05,height_shift_range=0.05,
                            zoom_range=0.05,horizontal_flip=True,vertical_flip=False)

# Calculation of necessary internal data for all images.
datagen.fit(X_train)

# Training

In [ ]:
model = get_model()
model.summary()

In [ ]:
# Optimizer (Adam Optimizer)
adam = Adam(lr = LR)

# Callbacks Declared
tensorboard = TensorBoard(log_dir="logs/{}".format(time()),batch_size=BATCH_SIZE)
# Supported in new version of keras ,update_freq='epoch')
reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.3,patience=3,verbose=1,
                              mode='max', min_lr=0.000001)
early_stop = EarlyStopping(monitor='val_loss',patience=3,verbose=1,mode='min')
# Supported in new version of keras ,restore_best_weights=True)
model_checkpoint = ModelCheckpoint(filepath=MODEL_PATH,monitor='val_acc',verbose=1,save_best_only=True,
                                  mode='max',period=3)

model.compile(optimizer = adam,loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
# Toggle if dont want to train using Image Augmentation
generator_train = True
EPOCHS = 30
callbacks=[tensorboard,reduce_lr,early_stop,model_checkpoint]

if generator_train:
    print(f'Training model {MODEL_NAME} using Image Augmentation')
    hist = model.fit_generator(datagen.flow(X_train,y_train,batch_size=BATCH_SIZE),
                               steps_per_epoch=len(X_train)//BATCH_SIZE,epochs=EPOCHS,verbose=2,
                               validation_data=(X_val,y_val),callbacks=callbacks)
else:
    print(f'Training model {MODEL_NAME} using normal image data provided')
    hist = model.fit(X_train,y_train,epochs=EPOCHS,batch_size=BATCH_SIZE,validation_data=(X_val,y_val),
                     verbose=2,callbacks=callbacks)

# Testing

In [ ]:
test_data = get_testing_data()

In [ ]:
X_test = np.array([i[0] for i in test_data]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
ids = [i[1] for i in test_data]

In [ ]:
pred = model.predict(X_test)